This Notebook is only designed for testing of correct alignment of the json files as well as to create anonymized emails based on the json files for testing purposes.

In [15]:
import json
import spacy
from spacy.training import offsets_to_biluo_tags
from spacy.lang.de import German
from pathlib import Path

def validate_alignment(file_path):
    nlp = German()
    file_path = Path(file_path)

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    total_files = len(data)
    misaligned_files = 0
    misaligned_details = []

    for entry in data:
        text = entry["text"]
        labels_raw = entry.get("labels", [])
        labels = [(l["start"], l["end"], l["label"]) for l in labels_raw]
        doc = nlp.make_doc(text)

        try:
            tags = offsets_to_biluo_tags(doc, labels)
        except Exception:
            continue

        if "-" in tags:
            misaligned_files += 1
            details = []
            for l in labels:
                s, e, t = l
                original = text[s:e].replace("\n", "\\n")
                details.append(f"{t}: '{original}' @ {s}-{e}")
            misaligned_details.append((entry.get("file", "unknown"), details))

    print("\n🔍 Ergebnis:")
    print(f"  Insgesamt überprüft: {total_files}")
    print(f"  Davon mit fehlerhaften Labels: {misaligned_files}\n")

    # Nur zusammengefasste Ausgabe:
   # for filename, issues in misaligned_details:
      #  print(f"⚠️ Datei: {filename}")
      #  for issue in issues:
       #     print(f"  → {issue}")
        #print()

# Beispielaufruf
validate_alignment("/ground_truth_split/train_norm.json")

FileNotFoundError: [Errno 2] No such file or directory: '/ground_truth_split/train_norm.json'

In [2]:
#Test zwei

import json

def validate_spans(path):
    errors = []
    with open(path, encoding="utf‑8") as f:
        data = json.load(f)
    # Stelle sicher, dass es eine Liste ist
    if isinstance(data, dict):
        data = [data]
    for idx, entry in enumerate(data):
        text = entry["text"]
        for lab in entry["labels"]:
            start, end, label = lab["start"], lab["end"], lab["label"]
            # 1) Out‑of‑bounds?
            if start < 0 or end > len(text) or start >= end:
                errors.append(
                    f"Beispiel {idx}: Ungültige Indizes für '{label}': "
                    f"start={start}, end={end}, Textlänge={len(text)}"
                )
                continue
            span = text[start:end]
            # 2) Leerzeichen am Rand?
            if span[0].isspace() or span[-1].isspace():
                errors.append(
                    f"Beispiel {idx}: Label '{label}' spannt auf whitespace: '{span!r}'"
                )
            # 3) Sehr einfacher Check auf Satzzeichen am Rand
            if span[0] in ".,;:!?" or span[-1] in ".,;:!?":
                errors.append(
                    f"Beispiel {idx}: Label '{label}' spannt auf Satzzeichen: '{span!r}'"
                )
    if errors:
        print("⚠️ Gefundene Probleme:")
        for e in errors[:20]:  # zeige max. 20 Errors
            print(" -", e)
        print(f"Insgesamt {len(errors)} Probleme gefunden.")
    else:
        print("✅ Alle Annotationen scheinen korrekt indiziert zu sein.")

# Aufruf für deine Datei
validate_spans("synthetic_mails_option_a_cleaned_new.json")

⚠️ Gefundene Probleme:
 - Beispiel 0: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''406976889822.''
 - Beispiel 1: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''405563747587.''
 - Beispiel 2: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''408492564592.''
 - Beispiel 2: Label 'STRASSE' spannt auf Satzzeichen: ''Schweitzerstr.''
 - Beispiel 3: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''409322946798.''
 - Beispiel 4: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''400817607062.''
 - Beispiel 5: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''407 445 316 632.''
 - Beispiel 6: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''400 968 612 176.''
 - Beispiel 7: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''403 964 781 803.''
 - Beispiel 8: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''408 503 214 898.''
 - Beispiel 9: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''405892097952.''
 - Beispiel 10: Label 'VERTRAGSNUMMER' spannt auf Satzzeichen: ''405 661 758 188.''
 - Beis

In [9]:
import json
import spacy
from spacy.training import offsets_to_biluo_tags
from spacy.lang.de import German
from pathlib import Path

def validate_alignment(file_path):
    nlp = German()
    file_path = Path(file_path)

    # 1) Überprüfe zuerst, ob die Datei überhaupt existiert
    if not file_path.exists():
        print(f"❌ Datei nicht gefunden: {file_path.resolve()}")
        return

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    total_files = len(data)
    misaligned_files = 0
    misaligned_details = []

    for entry in data:
        text = entry.get("text", "")
        labels_raw = entry.get("labels", [])
        labels = [(l["start"], l["end"], l["label"]) for l in labels_raw]
        doc = nlp.make_doc(text)

        try:
            tags = offsets_to_biluo_tags(doc, labels)
        except Exception as e:
            # 2) Gib die Exception-Meldung und den komplexen Kontext aus
            print(f"\n⚠️ Exception in offsets_to_biluo_tags für Datei '{entry.get('file','<kein dateiname>')}'")
            print("  Error:", e)
            print("  Text-Ausschnitt:", repr(text[:50]), "…")
            print("  Labels:", labels)
            # optional: springe hier nicht ab, sondern schaue dir das nächste Beispiel an
            continue

        # 3) Wenn BILUO-Tags ein '-' enthalten, dann ist Alignment futsch
        if any(tag == "-" for tag in tags):
            misaligned_files += 1
            details = []
            for start, end, label in labels:
                span_text = text[start:end].replace("\n", "\\n")
                details.append(f"{label}: '{span_text}' @ {start}-{end}")
                # 4) Zusätzlich prüfen, ob doc.char_span überhaupt eine gültige Span erzeugt
                span = doc.char_span(start, end, label=label, alignment_mode="strict")
                if span is None:
                    details.append(f"   → doc.char_span konnte '{span_text}' nicht finden")
            misaligned_details.append((entry.get("file", "unknown"), details))

    print("\n🔍 Ergebnis:")
    print(f"  Insgesamt überprüft: {total_files}")
    print(f"  Davon mit fehlerhaften Labels: {misaligned_files}\n")

    # 5) Ausführliche Details ausgeben
    for filename, issues in misaligned_details:
        print(f"⚠️ Datei: {filename}")
        for issue in issues:
            print(f"  → {issue}")
        print()

# Beispielaufruf
validate_alignment("synthetic_mails_option_a_cleaned_new.json")


🔍 Ergebnis:
  Insgesamt überprüft: 13560
  Davon mit fehlerhaften Labels: 0



In [11]:
import json
from pathlib import Path


def anonymize_file(input_path, output_path):
    # Lade die JSON-Datei
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    anonymized_entries = []

    for entry in data:
        text = entry["text"]
        # Ersetze Labels von hinten nach vorne – kein Offset nötig!
        labels = sorted(entry["labels"], key=lambda l: l["start"], reverse=True)

        for label in labels:
            start, end = label["start"], label["end"]
            placeholder = f"[{label['label']}]"
            text = text[:start] + placeholder + text[end:]

        anonymized_entries.append({
            "file": entry["file"],
            "anonymized_text": text
        })

    # Speichere das Ergebnis als neue JSON-Datei
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(anonymized_entries, f, ensure_ascii=False, indent=2)

    print(f"✅ Anonymisierte Datei gespeichert unter: {output_path.resolve()}")

# Beispiel: Nutze den tatsächlichen Dateinamen, z. B. "granu_train.json"
input_file = Path("./golden_dataset_norm_cleaned.json")  # 👉 Passe das an deine hochgeladene Datei an
output_file = Path("./anonymized_output_cleaned.json")

anonymize_file(input_file, output_file)

✅ Anonymisierte Datei gespeichert unter: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/Desktop/Veranstaltungen/Data Analytics in Applications/daia-eon/data/original/anonymized_output_cleaned.json
